# Capacitance matrix and LOM analysis
### Prerequisite
You need to have a working local installation of Ansys.

## 1. Create the design in Metal

In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
import qiskit_metal as metal
from qiskit_metal import designs, draw
from qiskit_metal import MetalGUI, Dict, Headings

In [3]:
design = designs.DesignPlanar()
gui = MetalGUI(design)

from qiskit_metal.qlibrary.qubits.transmon_pocket import TransmonPocket
from qiskit_metal.qlibrary.tlines.meandered import RouteMeander

In [4]:
design.variables['cpw_width'] = '15 um'
design.variables['cpw_gap'] = '9 um'

### In this example, the design consists of 4 qubits and 4 CPWs

In [5]:
# Allow running the same cell here multiple times to overwrite changes
design.overwrite_enabled = True

## Custom options for all the transmons
options = dict(
    # Some options we want to modify from the defaults
    # (see below for defaults)
    pad_width = '425 um', 
    pocket_height = '650um',
    # Adding 4 connectors (see below for defaults)
    connection_pads=dict(
        readout = dict(loc_W=+1,loc_H=-1, pad_width='200um'),
        bus1 = dict(loc_W=-1,loc_H=+1, pad_height='30um'),
        bus2 = dict(loc_W=-1,loc_H=-1, pad_height='50um')
    )
)

## Create 4 transmons

q1 = TransmonPocket(design, 'Q1', options = dict(
    pos_x='+2.42251mm', pos_y='+0.0mm', **options))
q2 = TransmonPocket(design, 'Q2', options = dict(
    pos_x='+0.0mm', pos_y='-0.95mm', orientation = '270', **options))
q3 = TransmonPocket(design, 'Q3', options = dict(
    pos_x='-2.42251mm', pos_y='+0.0mm', orientation = '180', **options))
q4 = TransmonPocket(design, 'Q4', options = dict(
    pos_x='+0.0mm', pos_y='+0.95mm', orientation = '90', **options))

RouteMeander.get_template_options(design)

options = Dict(
        lead=Dict(
            start_straight='0.2mm',
            end_straight='0.2mm'),
        trace_gap='9um',
        trace_width='15um')

def connect(component_name: str, component1: str, pin1: str, component2: str, pin2: str,
            length: str, asymmetry='0 um', flip=False, fillet='90um'):
    """Connect two pins with a CPW."""
    myoptions = Dict(
        fillet=fillet,
        hfss_wire_bonds = True,
        pin_inputs=Dict(
            start_pin=Dict(
                component=component1,
                pin=pin1),
            end_pin=Dict(
                component=component2,
                pin=pin2)),
        total_length=length)
    myoptions.update(options)
    myoptions.meander.asymmetry = asymmetry
    myoptions.meander.lead_direction_inverted = 'true' if flip else 'false'
    return RouteMeander(design, component_name, myoptions)

asym = 140
cpw1 = connect('cpw1', 'Q1', 'bus2', 'Q2', 'bus1', '6.0 mm', f'+{asym}um')
cpw2 = connect('cpw2', 'Q3', 'bus1', 'Q2', 'bus2', '6.1 mm', f'-{asym}um', flip=True)
cpw3 = connect('cpw3', 'Q3', 'bus2', 'Q4', 'bus1', '6.0 mm', f'+{asym}um')
cpw4 = connect('cpw4', 'Q1', 'bus1', 'Q4', 'bus2', '6.1 mm', f'-{asym}um', flip=True)

gui.rebuild()
gui.autoscale()

## 2. Capacitance Analysis and LOM derivation using the analysis package - most users

### Capacitance Analysis
Select the analysis you intend to run from the `qiskit_metal.analyses` collection.<br>
Select the design to analyze and the tool to use for any external simulation

In [6]:
from qiskit_metal.analyses.quantization import LOManalysis
c1 = LOManalysis(design, "q3d")

(optional) You can review and update the Analysis default setup following the examples in the next two cells.

In [7]:
c1.sim.setup

{'name': 'Setup',
 'reuse_selected_design': True,
 'reuse_setup': True,
 'freq_ghz': 5.0,
 'save_fields': False,
 'enabled': True,
 'max_passes': 15,
 'min_passes': 2,
 'min_converged_passes': 2,
 'percent_error': 0.5,
 'percent_refinement': 30,
 'auto_increase_solution_order': True,
 'solution_order': 'High',
 'solver_type': 'Iterative'}

In [8]:
# example: update single setting
c1.sim.setup.max_passes = 6
# example: update multiple settings
c1.sim.setup_update(solution_order = 'Medium', auto_increase_solution_order = 'False')

c1.sim.setup

{'name': 'Setup',
 'reuse_selected_design': True,
 'reuse_setup': True,
 'freq_ghz': 5.0,
 'save_fields': False,
 'enabled': True,
 'max_passes': 6,
 'min_passes': 2,
 'min_converged_passes': 2,
 'percent_error': 0.5,
 'percent_refinement': 30,
 'auto_increase_solution_order': 'False',
 'solution_order': 'Medium',
 'solver_type': 'Iterative'}

Analyze a single qubit with 2 endcaps using the default (or edited) analysis setup. Then show the capacitance matrix (from the last pass).

You can use the method `run()` instead of `sim.run()` in the following cell if you want to run both cap extraction and lom analysis in a single step. If so, make sure to also tweak the setup for the lom analysis. The input parameters are otherwise the same for the two methods. 

In [27]:
c1.sim.renderer.stop()

PyAEDT INFO: Desktop has been released and closed.


INFO:Global:Desktop has been released and closed.


True

In [9]:
c1.sim.run(components=['Q1'], open_terminations=[('Q1', 'readout'), ('Q1', 'bus1'), ('Q1', 'bus2')])
c1.sim.capacitance_matrix

PyAEDT INFO: Python version 3.11.13 (main, Aug  8 2025, 17:07:28) [Clang 20.1.4 ].


INFO:Global:Python version 3.11.13 (main, Aug  8 2025, 17:07:28) [Clang 20.1.4 ].


PyAEDT INFO: PyAEDT version 0.23.0.


INFO:Global:PyAEDT version 0.23.0.


PyAEDT INFO: Initializing new Desktop session.


...

PyAEDT INFO: aedt file load time 0.011152267456054688


INFO:Global:aedt file load time 0.011152267456054688


PyAEDT INFO: Export Convergence to  /tmp/tmp8fxqmxm3CG.conv


INFO:Global:Export Convergence to  /tmp/tmp8fxqmxm3CG.conv


,bus1_connector_pad_Q1,bus2_connector_pad_Q1,ground_main_plane,pad_bot_Q1,pad_top_Q1,readout_connector_pad_Q1
bus1_connector_pad_Q1,49.819171,-0.422431,-33.473712,-1.566193,-13.235570,-0.205103
bus2_connector_pad_Q1,-0.422431,53.944394,-35.813746,-13.867350,-1.832315,-1.015482
ground_main_plane,-33.473712,-35.813746,237.802860,-31.527922,-37.959358,-36.571830
pad_bot_Q1,-1.566193,-13.867350,-31.527922,98.124430,-30.211632,-18.793359
pad_top_Q1,-13.235570,-1.832315,-37.959358,-30.211632,88.150563,-2.210790
readout_connector_pad_Q1,-0.205103,-1.015482,-36.571830,-18.793359,-2.210790,59.867686


(otional - case-dependent)<br>If the previous cell was interrupted due to license limitations and for any reason you finally manually launched the simulation from the renderer GUI (outside qiskit-metal) you might be able to recover the simulation results by uncommenting and executing the following cell

In [22]:
# c1.sim._get_results_from_renderer()
# c1.sim.capacitance_matrix

The last variables you pass to the `run()` or `sim.run()` methods, will be stored in the `sim.setup` dictionary under the key `run`. You can recall the information passed by either accessing the dictionary directly, or by using the print handle below.

In [10]:
# c1.setup.run    <- direct access
c1.sim.print_run_args()

This analysis object run with the following kwargs:
{'name': None, 'components': ['Q1'], 'open_terminations': [('Q1', 'readout'), ('Q1', 'bus1'), ('Q1', 'bus2')], 'box_plus_buffer': True}



You can re-run the analysis after varying the parameters.<br>
Not passing the parameter `components` to the `sim.run()` method, skips the rendering and tries to run the analysis on the latest design. If a design is not found, the full metal design is rendered.

In [30]:
c1.sim.setup.freq_ghz = 4.8
c1.sim.run()
c1.sim.capacitance_matrix

PyAEDT INFO: Python version 3.11.13 (main, Aug  8 2025, 17:07:28) [Clang 20.1.4 ].


INFO:Global:Python version 3.11.13 (main, Aug  8 2025, 17:07:28) [Clang 20.1.4 ].


PyAEDT INFO: PyAEDT version 0.23.0.


INFO:Global:PyAEDT version 0.23.0.


PyAEDT INFO: Initializing new Desktop session.


...

PyAEDT INFO: aedt file load time 0.18852972984313965


INFO:Global:aedt file load time 0.18852972984313965


PyAEDT INFO: Export Convergence to  /tmp/tmpdkgozfg9CG.conv


INFO:Global:Export Convergence to  /tmp/tmpdkgozfg9CG.conv


,bus1_connector_pad_Q1,bus1_connector_pad_Q2,bus1_connector_pad_Q3,bus1_connector_pad_Q4,g_wb,pad_bot_Q1,pad_bot_Q2,pad_bot_Q3,pad_bot_Q4,pad_top_Q1,pad_top_Q2,pad_top_Q3,pad_top_Q4
bus1_connector_pad_Q1,1002.026478,-2.785813,-0.191511,-0.986844,-965.987706,-1.574115,-0.069004,-0.003747,-13.479879,-12.995340,-0.100653,-0.005614,-2.035283
bus1_connector_pad_Q2,-2.785813,986.142700,-0.970405,-0.189894,-950.770278,-13.410760,-1.793701,-0.005621,-0.124029,-1.802537,-12.802416,-0.005478,-0.084873
bus1_connector_pad_Q3,-0.191511,-0.970405,1002.437174,-2.775481,-966.955032,-0.005475,-13.772197,-1.544029,-0.072116,-0.006686,-2.029624,-12.767943,-0.126733
bus1_connector_pad_Q4,-0.986844,-0.189894,-2.775481,986.669513,-950.240537,-0.004266,-0.085573,-13.732758,-1.715134,-0.003774,-0.075102,-1.850282,-13.125160
g_wb,-965.987706,-950.770278,-966.955032,-950.240537,4414.451443,-51.273473,-51.718937,-51.679723,-50.459010,-42.319528,-41.846301,-42.619486,-42.377621
pad_bot_Q1,-1.574115,-13.410760,-0.005475,-0.004266,-51.273473,94.618134,-0.003424,-0.000643,-0.005093,-28.327387,-0.006560,-0.000743,-0.006194
pad_bot_Q2,-0.069004,-1.793701,-13.772197,-0.085573,-51.718937,-0.003424,96.185538,-0.003477,-0.014800,-0.009244,-28.467097,-0.007124,-0.021444
pad_bot_Q3,-0.003747,-0.005621,-1.544029,-13.732758,-51.679723,-0.000643,-0.003477,96.303147,-0.006061,-0.000759,-0.009100,-28.466883,-0.008835
pad_bot_Q4,-13.479879,-0.124029,-0.072116,-1.715134,-50.459010,-0.005093,-0.014800,-0.006061,94.538323,-0.008596,-0.023033,-0.004109,-28.351394
pad_top_Q1,-12.995340,-1.802537,-0.006686,-0.003774,-42.319528,-28.327387,-0.009244,-0.000759,-0.008596,86.502494,-0.002627,-0.000862,-0.008057


In [11]:
type(c1.sim.capacitance_matrix)

pandas.core.frame.DataFrame

### Lumped oscillator model (LOM)

Using capacitance matrices obtained from each pass, save the many parameters of the Hamiltonian of the system. `get_lumped_oscillator()` operates on 4 setup parameters: <br><br>
Lj: float <br>
Cj: float <br>
fr: Union[list, float] <br>
fb: Union[list, float] <br>

In [12]:
type(c1.sim.capacitance_all_passes)

dict

In [13]:
c1.setup.junctions = Dict({'Lj': 12.31, 'Cj': 2})
c1.setup.freq_readout = 7.0
c1.setup.freq_bus = [6.0, 6.2]

c1.run_lom()
c1.lumped_oscillator_all

02:20AM 51s INFO [run_lom]: Per-pass capacitance matrices not available. Using final capacitance matrix.


[3, 4] [5 0 1]
Predicted Values

Transmon Properties
f_Q 5.419121 [GHz]
EC 311.261754 [MHz]
EJ 13.273404 [GHz]
alpha -362.874773 [MHz]
dispersion 45.611653 [KHz]
Lq 12.305036 [nH]
Cq 62.231314 [fF]
T1 35.937855 [us]

**Coupling Properties**

tCqbus1 7.378436 [fF]
gbus1_in_MHz 113.980023 [MHz]
χ_bus1 -3.131318 [MHz]
1/T1bus1 2771.368526 [Hz]
T1bus1 57.428286 [us]

tCqbus2 -6.477979 [fF]
gbus2_in_MHz -85.977543 [MHz]
χ_bus2 -9.828658 [MHz]
1/T1bus2 1184.138291 [Hz]
T1bus2 134.405706 [us]

tCqbus3 5.335204 [fF]
gbus3_in_MHz 73.127846 [MHz]
χ_bus3 -4.375138 [MHz]
1/T1bus3 473.108772 [Hz]
T1bus3 336.402436 [us]
Bus-Bus Couplings
gbus1_2 7.114563 [MHz]
gbus1_3 9.888708 [MHz]
gbus2_3 5.359327 [MHz]


,EC,EJ,alpha,chi_in_MHz,dispersion,fQ,gbus,χr MHz,gr MHz
1,311.261754,13.273404,-362.874773,"[-3.1313176702596994, -9.828658365110826, -4.3...",45.611653,5.419121,"[113.98002267461709, -85.97754295125037, 73.12...",3.131318,113.980023


In [14]:
c1.plot_convergence();
c1.plot_convergence_chi()

Design "Design_q3d" info:
	# eigenmodes    0
	# variations    1


Once you are done with your analysis, please close it with `close()`. This will free up resources currently occupied by qiskit-metal to communiate with the tool.

In [15]:
c1.sim.close()

PyAEDT INFO: Desktop has been released and closed.


INFO:Global:Desktop has been released and closed.


## 3. Directly access the renderer to modify other parameters

In [ ]:
c1.sim.start()
c1.sim.renderer

PyAEDT INFO: Python version 3.11.13 (main, Aug  8 2025, 17:07:28) [Clang 20.1.4 ].


INFO:Global:Python version 3.11.13 (main, Aug  8 2025, 17:07:28) [Clang 20.1.4 ].


PyAEDT INFO: PyAEDT version 0.23.0.


INFO:Global:PyAEDT version 0.23.0.


PyAEDT INFO: Initializing new Desktop session.


...

INFO:Global:Log on AEDT is disabled.


PyAEDT INFO: Starting new AEDT gRPC session.


INFO:Global:Starting new AEDT gRPC session.


PyAEDT INFO: AEDT installation Path /ansys_inc/v252/AnsysEM


INFO:Global:AEDT installation Path /ansys_inc/v252/AnsysEM


Every renderer will have its own collection of methods. Below an example with q3d

##### Prepare and run a collection of predefined setups

This is equivalent to going to the Project Manager panel in Ansys, right clicking on Analysis within the active Q3D design, selecting "Add Solution Setup...", and choosing/entering default values in the resulting popup window. You might want to do this to keep track of different solution setups, giving each of them a different/specific name.

In [ ]:
setup = c1.sim.renderer.new_ansys_setup(name = "Setup_demo", max_passes = 6)

You can directly pass to `new_ansys_setup` all the setup parameters. Of course you will then need to run the individual setups by name as well.

In [ ]:
c1.sim.renderer.analyze_setup(setup.name)

##### Get the capactiance matrix at a different pass

You might want to use this if you intend to know what was the matrix at a different pass of the simulation.

In [ ]:
# Using the analysis results, get its capacitance matrix as a dataframe.
c1.sim.renderer.get_capacitance_matrix(variation = '', solution_kind = 'AdaptivePass', pass_number = 5)

### Code to swap rows and columns in capacitance matrix
from qiskit_metal.analyses.quantization.lumped_capacitive import df_reorder_matrix_basis

df_reorder_matrix_basis(fourq_q3d.get_capacitance_matrix(), 1, 2)

##### Close the renderer

In [ ]:
c1.sim.close()